In [1]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('../../Data/train.csv')
test_df = pd.read_csv('../../Data/test.csv')

# data overview

In [25]:
print(train_df.shape)
train_df.head()

(7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [24]:
print('target == 1 Case:')
train_df[train_df.target == 1]['text'].values[0]

target == 1 Case:


'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [23]:
print('target == 0 case:')
train_df[train_df.target == 0]['text'].values[0]

target == 0 case:


"What's up man?"

# basic statistic

In [30]:
train_df.groupby('target').count()

,id,keyword,location,text
target,,,,
0,4342,4323,2884,4342
1,3271,3229,2196,3271
